# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'BreastCancer-Experiment'

experiment=Experiment(ws, experiment_name)


# choose a name for experiment

project_folder = './breastcancer'
experiment

Name,Workspace,Report Page,Docs Page
BreastCancer-Experiment,quick-starts-ws-251697,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "azurecapstone-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)



InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded...........................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [25]:
# NOTE: update the key to match the dataset name
found = False
key = "BreastCancerDataset"
description_text = "Dataset for Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/samfrost23/nd00333-capstone/master/breast-cancer-dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.00,0.12,0.28,0.30,0.15,...,25.38,17.33,184.60,2019.00,0.16,0.67,0.71,0.27,0.46,0.12
1,842517,M,20.57,17.77,132.90,1326.00,0.08,0.08,0.09,0.07,...,24.99,23.41,158.80,1956.00,0.12,0.19,0.24,0.19,0.28,0.09
2,84300903,M,19.69,21.25,130.00,1203.00,0.11,0.16,0.20,0.13,...,23.57,25.53,152.50,1709.00,0.14,0.42,0.45,0.24,0.36,0.09
3,84348301,M,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11,...,14.91,26.50,98.87,567.70,0.21,0.87,0.69,0.26,0.66,0.17
4,84358402,M,20.29,14.34,135.10,1297.00,0.10,0.13,0.20,0.10,...,22.54,16.67,152.20,1575.00,0.14,0.20,0.40,0.16,0.24,0.08


In [6]:
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# Automl settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric" : 'AUC_weighted'
}

# Automl config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             enable_onnx_compatible_models=True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [8]:
# Submit experiment
from azureml.widgets import RunDetails
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
BreastCancer-Experiment,AutoML_cd384120-893a-45a8-8048-8d6fc6280a44,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
BreastCancer-Experiment,AutoML_cd384120-893a-45a8-8048-8d6fc6280a44,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_cd384120-893a-45a8-8048-8d6fc6280a44',
 'target': 'azurecapstone-project',
 'status': 'Completed',
 'startTimeUtc': '2024-02-05T09:09:36.949305Z',
 'endTimeUtc': '2024-02-05T09:36:25.120981Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'azurecapstone-project',
  'AMLSettingsJsonString': '{"path":null,"name":"BreastCancer-Experiment","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-251697","workspace_name":"quick-starts-ws-251697","region":"westus2","comput

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
automl_best_run, automl_best_model = remote_run.get_output()
automl_best_run


automl_best_model


Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('svcwrapper', SVCWrapper(C=51.79474679231202, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001, verbose=False))], ver

In [11]:
metrics = automl_best_run.get_metrics()
metrics['accuracy']

0.9841173730787144

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
# Save the best model
import joblib
from azureml.core.model import Model

description = "Best-fit AutoML model trained on Breast Cancer dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(automl_best_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-breastcancer', description=description)

In [15]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict Diagnosis - Malignant(M) or Benign(B)',
    auth_enabled=True
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:

from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'automl-breastcancer')


myenv = automl_best_run.get_environment()
entry_script = 'predict.py'
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-05 09:44:52+00:00 Creating Container Registry if not exists..
2024-02-05 09:54:52+00:00 Registering the environment.
2024-02-05 09:54:53+00:00 Generating deployment configuration.
2024-02-05 09:54:53+00:00 Submitting deployment to compute..
2024-02-05 09:54:59+00:00 Checking the status of deployment automl-breastcancer..
2024-02-05 09:58:37+00:00 Checking the status of inference endpoint automl-breastcancer.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [17]:
if service.enable_app_insights==False:
    service.update(enable_app_insights=True)
print(service.get_logs())

2024-02-05T09:58:27,585199700+00:00 - rsyslog/run 
2024-02-05T09:58:27,596851800+00:00 - gunicorn/run 
2024-02-05T09:58:27,601885000+00:00 | gunicorn/run | 
2024-02-05T09:58:27,608139300+00:00 | gunicorn/run | ###############################################
2024-02-05T09:58:27,615096700+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-05T09:58:27,618160600+00:00 - nginx/run 
2024-02-05T09:58:27,622123500+00:00 | gunicorn/run | ###############################################
2024-02-05T09:58:27,631830200+00:00 | gunicorn/run | 
2024-02-05T09:58:27,650066600+00:00 | gunicorn/run | 
2024-02-05T09:58:27,680495400+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-05T09:58:27,682532100+00:00 | gunicorn/run | 
2024-02-05T09:58:27,688912600+00:00 | gunicorn/run | 
2024-02-05T09:58:27,690772000+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [18]:
print("State: "+service.state)

State: Healthy


In [23]:
import requests
import json

scoring_uri = service.scoring_uri
key = service.get_keys()[0]

data = {
    "data":[
        {
            "id": 0,
            "radius_mean": 19.81,
            "texture_mean": 22.6,
            "perimeter_mean": 130,
            "area_mean": 1200.9,
            "smoothness_mean": 0.0986,
            "compactness_mean": 0.1062,
            "concavity_mean": 0.08015,
            "concave points_mean": 0.03799,
            "symmetry_mean": 0.189,
            "fractal_dimension_mean": 0.064040,
            "radius_se": 0.389,
            "texture_se": 1.0437,
            "perimeter_se": 2.499,
            "area_se": 30.26,
            "smoothness_se": 0.006989,
            "compactness_se": 0.01902,
            "concavity_se": 0.02722,
            "concave points_se": 0.01029,
            "symmetry_se": 0.01767,
            "fractal_dimension_se": 0.003596,
            "radius_worst": 14.85,
            "texture_worst": 24.71,
            "perimeter_worst": 96.01,
            "area_worst": 677.8,
            "smoothness_worst": 0.1415,
            "compactness_worst": 0.2354,
            "concavity_worst": 0.2680,
            "concave points_worst": 0.1016,
            "symmetry_worst": 0.3011,
            "fractal_dimension_worst": 0.0880
            
        },
        {
            "id": 0,
            "radius_mean": 18.88,
            "texture_mean": 21.29,
            "perimeter_mean": 123.55,
            "area_mean": 1129,
            "smoothness_mean": 0.09011,
            "compactness_mean": 0.1031,
            "concavity_mean": 0.109,
            "concave points_mean": 0.07953,
            "symmetry_mean": 0.1581,
            "fractal_dimension_mean": 0.05459,
            "radius_se": 0.7876,
            "texture_se": 0.7969,
            "perimeter_se": 5.479,
            "area_se": 96.03,
            "smoothness_se": 0.004433,
            "compactness_se": 0.01651,
            "concavity_se": 0.02259,
            "concave points_se": 0.0136,
            "symmetry_se": 0.01383,
            "fractal_dimension_se": 0.001697,
            "radius_worst": 24.84,
            "texture_worst": 26.57,
            "perimeter_worst": 165.7,
            "area_worst": 1855,
            "smoothness_worst": 0.1191,
            "compactness_worst": 0.2322,
            "concavity_worst": 0.2676,
            "concave points_worst": 0.1779,
            "symmetry_worst": 0.2553,
            "fractal_dimension_worst": 0.06576            
        }
    ]
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("The following are the results from the test (B=Benign, M=Malignangt):")
print(resp.json())

The following are the results from the test (B=Benign, M=Malignangt):
['B', 'M']


In [24]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx')

In [ ]:
service.delete()
compute_target.delet()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
